In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle

In [2]:
# Load the dataset
df = pd.read_csv("CAR DETAILS.csv")
df.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
df.shape

(4340, 8)

In [4]:
# Drop duplicates
df.drop_duplicates(inplace=True)
df.shape

(3577, 8)

In [5]:
# Calculate car age
df["car_age"] = 2023 - df["year"]
df.columns


Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'car_age'],
      dtype='object')

In [6]:
name = df["name"].str.split(" ", expand = True)
df["car_maker"] = name[0]
df["car_model"] = name[1]

In [7]:
df.drop(["name"], axis=1, inplace=True)

In [8]:
df.columns

Index(['year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'car_age', 'car_maker', 'car_model'],
      dtype='object')

In [9]:
# Encoding data with get dummies
df = pd.get_dummies(df, drop_first=True, columns=df.columns.difference(['selling_price', 'km_driven', 'year','car_age']))
df.head()

,year,selling_price,km_driven,car_age,car_maker_Audi,car_maker_BMW,car_maker_Chevrolet,car_maker_Daewoo,car_maker_Datsun,car_maker_Fiat,...,fuel_Electric,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual
0,2007,60000,70000,16,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,True
1,2007,135000,50000,16,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,False,True
2,2012,600000,100000,11,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
3,2017,250000,46000,6,False,False,False,False,True,False,...,False,False,True,False,False,False,False,True,False,True
4,2014,450000,141000,9,False,False,False,False,False,False,...,False,False,False,False,True,False,False,True,False,True


In [10]:
# Encode categorical columns
encoder = LabelEncoder()
df1 = df.apply(encoder.fit_transform)
df1.head()

,year,selling_price,km_driven,car_age,car_maker_Audi,car_maker_BMW,car_maker_Chevrolet,car_maker_Daewoo,car_maker_Datsun,car_maker_Fiat,...,fuel_Electric,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner,seller_type_Individual,seller_type_Trustmark Dealer,transmission_Manual
0,13,17,474,13,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
1,13,60,335,13,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
2,18,236,607,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,23,114,309,3,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,1
4,20,193,681,6,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1


In [11]:
# Split data into features and target
X = df1.drop("selling_price", axis=1)
y = df1["selling_price"]

In [12]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [13]:
# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "KNeighbors Regressor": KNeighborsRegressor(),
    "Random Forest Regressor": RandomForestRegressor()
}

In [14]:
df1.columns

Index(['year', 'selling_price', 'km_driven', 'car_age', 'car_maker_Audi',
       'car_maker_BMW', 'car_maker_Chevrolet', 'car_maker_Daewoo',
       'car_maker_Datsun', 'car_maker_Fiat',
       ...
       'fuel_Electric', 'fuel_LPG', 'fuel_Petrol',
       'owner_Fourth & Above Owner', 'owner_Second Owner',
       'owner_Test Drive Car', 'owner_Third Owner', 'seller_type_Individual',
       'seller_type_Trustmark Dealer', 'transmission_Manual'],
      dtype='object', length=227)

In [15]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_models(models, X_train, y_train, X_test, y_test):
    for name, model in models.items():
        print(f"Evaluating {name}:")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print("MAE:", mae)
        print("MSE:", mse)
        print("R2 Score:", r2)
        print("Training Score: ", model.score(X_train, y_train))
        print("Testing Score: ", model.score(X_test, y_test))
        print()
        
# Call the function to evaluate models        
evaluate_models(models, X_train, y_train, X_test, y_test)

Evaluating Linear Regression:
MAE: 257944032.4231508
MSE: 9.009106538158205e+18
R2 Score: -883447848407726.6
Training Score:  0.8989693945637564
Testing Score:  -883447848407726.6

Evaluating Ridge Regression:
MAE: 27.87113858960886
MSE: 1384.5049496907425
R2 Score: 0.8642331607764303
Training Score:  0.8951364310380663
Testing Score:  0.8642331607764303

Evaluating Lasso Regression:
MAE: 44.98835628983222
MSE: 3337.6174959621794
R2 Score: 0.6727077226662945
Training Score:  0.7060798511236004
Testing Score:  0.6727077226662945

Evaluating KNeighbors Regressor:
MAE: 52.837430167597766
MSE: 5130.409497206703
R2 Score: 0.4969035816636669
Training Score:  0.6628422470026722
Testing Score:  0.4969035816636669

Evaluating Random Forest Regressor:
MAE: 30.243753563669415
MSE: 1756.4037585397273
R2 Score: 0.8277641501024594
Training Score:  0.9774711325090644
Testing Score:  0.8277641501024594



In [ ]:
###  using hyper-perameter tuingng Random Forest Regressionrandom_state = 7, max_depth=10
rf = RandomForestRegressor(max_depth=15, random_state = 7)
rf.fit(X_train, y_train)

In [ ]:
def scores(ytest, ypred):
  mae = mean_absolute_error(ytest, ypred)
  mse = mean_squared_error(ytest, ypred)
  rmse = np.sqrt(mean_squared_error(ytest, ypred))
  r2 = r2_score(ytest, ypred)
  print("MAE: ", mae)
  print("MSE: ", mse)
  print("RMSE: ", rmse)
  print("R2 Score: ", r2)

def model_score(model):
  print("Training Score: ", model.score(xtrain, ytrain))
  print("Testing Score: ", model.score(xtest, ytest))

In [ ]:
### Saving the best model using Pickle
with open('rfmodel.pkl', 'wb') as file:
    pickle.dump(rf, file)


In [ ]:
df_sample = df1.sample(20)
df_sample.head()


In [ ]:
# Saving the Sample Dataset
df_sample.to_csv("sample-dataset.csv")

In [ ]:
### Selecting 20 Dependent and Independent Variable randomly from dataset

x_sample = df_sample.drop("selling_price", axis = 1)
y_sample = df_sample["selling_price"]
print(x_sample.shape)
print(y_sample.shape)

In [ ]:
### Splitting the data into Training and Testing from randomly selected dataset

xtrain, xtest, ytrain, ytest = train_test_split(x_sample, y_sample, test_size=.20,random_state=7)
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)

In [ ]:
#### Loading the best model using Pickle

# Load the saved trained ML model
with open('rfmodel.pkl', 'rb') as file:
    best_model_sample = pickle.load(file)
best_model_sample

In [ ]:
best_model_sample.fit(xtrain,ytrain)

In [ ]:
#### Generate the Predictions for randomly 20 points selected from dataset

best_model_sample.fit(xtrain,ytrain)

In [ ]:
### Checking the Scores for randomly 20 points selected from dataset
model_score(rf)

ypred_rf = rf.predict(xtest)
ypred_rf

scores(ytest, ypred_rf)